In [6]:
import requests, urllib.request
import json
import re, os
from bs4 import BeautifulSoup as bs
from selenium.webdriver import Chrome

In [7]:
delay=3
browser = Chrome()
browser.implicitly_wait(delay)

In [8]:
def blog_crawling(keyword, start, end):
    
    count = 0 # 이미지 저장 수
    
    folder_num = 0 # 이미지 저장 폴더 수

    results = []
    
    for i in range(1,400,10): # 네이버 검색은 최상의 검색결과 품질을 위해 1000건 이상의 블로그 검색결과를 제공하지 않는다.
                                      # 해당 키워드와 기간 설정 후 끝페이지 확인 후 그에 맞게 for문 설정
        
        base_url = 'https://search.naver.com/search.naver?where=post&query='
    
        period = '&st=sim&sm=tab_opt&date_from=' + start + '&date_to=' + end

        empt = '&date_option=8&srchby=all&dup_remove=1&post_blogurl=blog.naver.com&post_blogurl_without=&nso=so%3Ar%2Ca%3Aall%2Cp%3Afrom' + start + 'to' + end

        page_count = '&post_blogurl=blog.naver.com&post_blogurl_without=&query=다이슨&sm=tab_pge&srchby=all&st=sim&where=post&start=' + str(i)
        # page_count에 query='해당 키워드' 설정해야함
        search_url = base_url + keyword + period + empt + page_count
        
        response = requests.get(search_url) # 블로그 출처 blog.naver.com 설정된 url 주소

        soup = bs(response.text, 'html.parser')
        
        browser.get(search_url)
        
        
        for links in soup.select('li.sh_blog_top > dl'):
            
            blog_info = []
            
            post_addr = links.select('dd.txt_block a')
            
            post_url = post_addr[1].get('href')
            
            delay=3
            browser.get(post_url)
            browser.implicitly_wait(delay)
            
            browser.switch_to.frame('mainFrame') # 네이버 블로그 수집위해 frame 전환
            
            soup = bs(browser.page_source, 'html.parser')
            
            try: # img, img2 나눈 이유는 블로그마다 이미지 소스코드가 다르기 때문에
                img = browser.find_elements_by_css_selector('div > div > a > img.se-image-resource') 
                img2 = browser.find_elements_by_css_selector('div > div > p > img._photoImage')

                link = []

                for i in range(len(img)): # 해당 블로그 이미지 개수만큼 추가
                    link.append(img[i].get_attribute('src'))

                link2 = []

                for i in range(len(img2)): # 해당 블로그 이미지 개수만큼 추가
                    link2.append(img2[i].get_attribute('src'))

            except:
                img = soup.find_all("img")

                link = []

                for i in range(len(img)): # 해당 블로그 이미지 개수만큼 추가
                    link.append(img[i].get('src'))
                
            os.chdir('/Users/CPB06GameN/Desktop/와이즈넛 실무/네이버 블로그 크롤링/Crawling_미국쑥부쟁이/') # 이미지 저장 폴더 경로 설정
                             
            folder_num += 1

            try: # 폴더 생성
                os.makedirs(os.path.join('BLOG'+str(folder_num)))

            except OSError as e:
                if e.errno != errno.EEXIST:
                    print("폴더 생성 실패")
                    exit() 

            for url in link: # 이미지 1 파일 저장
                count+=1
                urllib.request.urlretrieve(url, 'BLOG'+str(folder_num)+'/'+'BLOG_img'+str(count)+'.jpg')  

            count = 0 # 이미지 저장명 초기화

            for url in link2: # 이미지 2 파일 저장
                count+=1
                urllib.request.urlretrieve(url, 'BLOG'+str(folder_num)+'/'+'BLOG_img'+str(count)+'.jpg') 

            count = 0 # 이미지 저장명 초기화   
            
            
            title_info = soup.find('meta', property = 'og:title') # 글 제목 추출
            
            title = title_info['content']
            
            try: # 블로그마다 소스코드가 달라서 2가지 경우의 예외처리 추가
                date = browser.find_element_by_css_selector('div > div > div.blog2_container > span.se_publishDate.pcol2').text # 글 쓴 날짜 및 시간 추출
                                
                content_tags = soup.select('div > div.se-main-container')[0].select('p') # 본문 내용 추출
                
                content_info = ' '.join([ tags.get_text() for tags in content_tags ])
            
                content = re.sub('\u200b','',content_info)
                
            except:
                
                try:
                    date = browser.find_element_by_css_selector('tbody > tr > td.bcc > table > tbody > tr > td > p.date.fil5.pcol2._postAddDate').text # 글 쓴 날짜 및 시간 추출
                
                    content_temp = soup.find("div", {"id": "postViewArea"}) # 본문 내용 추출

                    content_info = content_temp.text

                    content = re.sub('\u200b','',content_info)
                    
                except:
                    date = browser.find_element_by_css_selector('div > div > div.blog2_container > span.se_publishDate.pcol2').text # 글 쓴 날짜 및 시간 추출
                    
                    content = browser.find_element_by_css_selector('div.se_component_wrap.sect_dsc.__se_component_area').text # 본문 내용 추출



            blog_info = {
                            'title' : title,
                            'content' : content,
                            'date' : date            
                        }
            
            results.append(blog_info)
        
    return results

In [9]:
if __name__ == "__main__":

    results_Print = blog_crawling('미국쑥부쟁이','20190101','20191025')

    with open('naverBlog_crawling_data.json','w',encoding = 'utf-8') as make_file:
        json.dump(results_Print,make_file,ensure_ascii=False,indent='\t')


KeyboardInterrupt: 